In [1]:
import numpy as np
import itertools
import gzip
def read_fasta_chrom_pos(fasta_path, chrom, start, stop):
    """
    Extracts a subsequence from a chromosome in a FASTA file.
    """
    from Bio import SeqIO
    with open(fasta_path, "rt") as f:
        for record in SeqIO.parse(f, "fasta"):
            if record.id == chrom:
                return str(record.seq[start:stop])
    raise ValueError(f"Chromosome {chrom} not found in {fasta_path}")

In [2]:

%pip install biopython

#counts all k-mers in genomic region 
def kmer_count(fasta_path, chrom, start, stop, k=3):
    #read_fasta_chrom_pos temp variable
    if fasta_path.endswith(".gz"):
        with gzip.open(fasta_path, "rt") as f:
            sequence = read_fasta_chrom_pos(fasta_path, chrom, start, stop).upper()
    else:
        sequence = read_fasta_chrom_pos(fasta_path, chrom, start, stop).upper()
    
    counts = {}
    #enact sliding window of size k across sequence
    for i in range(len(sequence) - k + 1):
        kmer = sequence[i:i+k]
        if 'N' in kmer:  # Skip kmers with 'N' bases
            continue
        counts[kmer] = counts.get(kmer, 0) + 1
    return counts

counts = kmer_count("chr20.fa.gz", "chr20", 1000, 1100, k=5)
print(counts)  # Should show k-mer counts

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
{}


In [3]:
#background k-mer frequency distribution using random samples
def build_kmer_background(fasta_path, chrom, num_samples=1000, sample_size=100, k=3):
   
    background = {}
    total_counts = 0
    
    # Determine chromosome length
    chrom_sequence = read_fasta_chrom_pos(fasta_path, chrom, 0, 1_000_000)  # Adjust range as needed
    chrom_len = len(chrom_sequence)

    # using these embedding loops, for each region(1000) samples random postion
    for _ in range(num_samples):
        # Sample random position ensuring we don't go past chromosome end
        pos = np.random.randint(0, chrom_len - sample_size)
        counts = kmer_count(fasta_path, chrom, pos, pos + sample_size, k)
        
        for kmer, count in counts.items():
            background[kmer] = background.get(kmer, 0) + count
        total_counts += sum(counts.values())
    
    # Convert to frequencies
    for kmer in background:
        background[kmer] = background[kmer] / total_counts
    
    return background
#identify overrepresented k-mers in observed counts
#MMEJ resilience to short sequences (3-7bp)

In [4]:
def max_background_frequency(observed_counts, background_freq):
    significance = {}
    total_observed = sum(observed_counts.values())
    for kmer, obs_count in observed_counts.items():
        expected_freq = background_freq.get(kmer, 1e-6)  # avoid division by zero
        expected_count = expected_freq * total_observed
        # calculate observed vs expected frequency if freq>1 indicates overrepresentation
        significance[kmer] = obs_count / expected_count if expected_count != 0 else 0
    return significance

In [5]:
#look for homology per chromosome exclude n MMEJ model for probaility districbution for every chromosome.
# Every observation , normalize by V1(w1-k) = p(k=k1)
#dictionary of kmer counts with obsrvation 100bp ( to avid 0 enumerate kmers with 0 counts)
# expected value calculate
# P arm and q arm 
#keep keys and counts n= l-k m=l-k
#R1 and r2
# for the homology analysis, n = (start, stop, + 1 ) - k , L +=n , m= where l will keep track of each length
# 1/ 4k - k 
#markov property

def enrichment(fasta_path, chrom, start, stop, k=5, pseudocount=1e-3):
    """
    Computes k-mer enrichment scores for a genomic region, incorporating pseudocounts and region-length adjustments.
    
    Args:
        fasta_path: Path to genome FASTA.
        chrom: Chromosome (e.g., 'chr20').
        start: Start position.
        stop: End position.
        k: K-mer size.
        pseudocount: Small value to avoid zero counts (default=1e-3).
    
    Returns:
        Dict of k-mers with enrichment scores.
    """
    # Step 1: Calculate valid k-mers in the region (n = L - k)
    sequence = read_fasta_chrom_pos(fasta_path, chrom, start, stop)
    L = stop - start + 1
    n = L - k + 1  # Total possible k-mers
    
    # Step 2: Get observed k-mers with pseudocounts (advisor's m = {k:v} with no zeros)
    observed = kmer_count(fasta_path, chrom, start, stop, k)
    all_kmers = [''.join(p) for p in itertools.product('ACGT', repeat=k)]
    observed_with_pseudo = {kmer: observed.get(kmer, pseudocount) for kmer in all_kmers}
    
    # Step 3: Get background frequencies (advisor's M = {Ki[X,Y]})
    background = build_kmer_background(fasta_path, chrom, k=k)
    
    # Step 4: Normalize counts (advisor's m[a] - 1/|K| adjustment)
    total_kmers = len(all_kmers)
    normalized = {}
    for kmer, obs in observed_with_pseudo.items():
        # Adjust observed count by pseudocount and |K|
        adj_obs = obs - (pseudocount / total_kmers)  # Example: m[a] - 1/|K|
        expected = background.get(kmer, pseudocount) * n
        normalized[kmer] = adj_obs / expected if expected != 0 else 0
    
    return normalized

fasta_path = "chr20.fa.gz"  
chrom = "chr20"             
start, stop = 10000, 10500  # Test region
k = 5

# Run analysis
results = enrichment(fasta_path, chrom, start, stop, k)

# Print top 10 results
for kmer, score in sorted(results.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"{kmer}: {score:.2f}")

AAGCC: 0.01
AAGTT: 0.01
AATTG: 0.01
AATTT: 0.01
ACAAT: 0.01
ACACG: 0.01
ACAGT: 0.01
ACCCA: 0.01
ACTGT: 0.01
AGCCA: 0.01
